# KINTSUGI: Knowledge Integration with New Technologies for Simplified User-Guided Image processing

### In the following notebook you will use the blank channels to isolate signal.

## 1. Import packages. 
### *This must be done every time the notebook is started or restarted.

In [1]:
from skimage.io import imread, imshow, imsave
import os
import numpy as np
import Kview
from matplotlib import pyplot as plt
from skimage import data, io
from skimage import morphology
from skimage import filters
from skimage import measure
import pandas as pd
from glob import glob
from datetime import datetime
import xml.etree.ElementTree as ET
import tifffile
current_dateTime = datetime.now()
import warnings
warnings.filterwarnings('ignore')


def ini_params(images, imageb, Pixel_Cutoff:int=0, Blank_Percentile:float=0.0):

    blank_image = np.clip(imageb, Pixel_Cutoff, 65535)
    blank_image[blank_image <= Pixel_Cutoff] = 0
    
    signal_image = images - (np.minimum(images, blank_image * Blank_Percentile))
    
    return signal_image

def find_gauss(signal_image2, sigma_value:int=10):
    signal_image = signal_image2 * 1.01
    blank = filters.gaussian(signal_image, sigma=sigma_value, preserve_range=True)
    return blank

def clipped_blank(blank_image, Gauss_low:int = 0, Gauss_high:int = 65535):
    blank_image2 = blank_image * 1.01
    # blank2 = np.clip(blank_image, Gauss_low, np.max(blank_image))
    blank_image2[Gauss_low >= blank_image2] = 0  
    blank_image2[blank_image2 >= Gauss_high] = 0
    return blank_image2

def final_factor(sig_im, bl, low, high, Gauss_Percentile:float=0.2):

    bl[low >= bl] = 0  
    bl[bl >= high] = 0

    signal_im2 = sig_im - (np.minimum(sig_im, bl * Gauss_Percentile))
    return signal_im2

## 2. Define directory paths. 
### *This must be done every time the notebook is started or restarted.

In [4]:
reg_dir = "C:/Users/smith6jt/KINTSUGI/data/1904_CC2B_Registration/"
sig_dir = reg_dir.replace('_Registration', '_Signal_Isolation')
proc_dir = reg_dir.replace('_Signal_Isolation', '_Processed')
print(f"Registration folder is {reg_dir}.")
print(f"Signal_Isolation folder is {sig_dir}.")
print(f"Processed folder is {proc_dir}.")

Registration folder is C:/Users/smith6jt/KINTSUGI/data/1904_CC2B_Registration/.
Signal_Isolation folder is C:/Users/smith6jt/KINTSUGI/data/1904_CC2B_Signal_Isolation/.
Processed folder is C:/Users/smith6jt/KINTSUGI/data/1904_CC2B_Registration/.


### 2.1 This cell only needs to be run once.

In [10]:
os.makedirs(sig_dir, exist_ok=True)
os.makedirs(proc_dir, exist_ok=True)
os.makedirs(os.path.join(proc_dir,'Processing_parameters'), exist_ok=True)
os.makedirs(os.path.join(proc_dir,'ImageJ'), exist_ok=True)

## 3. Write single channel tifs

In [8]:
ome_tiff_path = reg_dir + "merged.ome.tif"
ome_tiff = tifffile.imread(ome_tiff_path)

with tifffile.TiffFile(ome_tiff_path) as tif:

    xml_description = tif.ome_metadata

root = ET.fromstring(xml_description)
namespace = {'ome': 'http://www.openmicroscopy.org/Schemas/OME/2016-06'}

channel_metadata = {}
channel_names = []
for channel in root.findall('.//ome:Channel', namespace):
    channel_name = channel.attrib['Name']
    metadata = {}
    for key, value in channel.attrib.items():
        if key != 'Name':
            metadata[key] = value
    channel_metadata[channel_name] = metadata
    channel_names.append(channel_name)


for channel_name, metadata in channel_metadata.items():

    channel_index = channel_names.index(channel_name)
    channel_data = ome_tiff[channel_index, ...]
    channel_image_path = os.path.join(sig_dir, f'{channel_name}.tif')
    imsave(channel_image_path, channel_data, plugin='tifffile', metadata=metadata)

## 4. Determine pixel range and weight of blank channel subtraction.

### 4.1 Instructions

This cell allows the user to choose which blank channel to use (BlankID) and determine how much, if any, blank image to subtract(Blank_Percentile).  It also allows for only subtracting a range of pixel values (Pixel_Cutoff).  Use these values in the next cell.
For each marker, enter the name of the image file without the extension for signal_channel.

Change 'bl_CH' to a, b, or c according to the channel that marker was imaged on i.e. if marker is CH3, the last letter of both blanks should be 'b', CH4 should be 'c'.  For far-red channels (700nm+), the blank channels often do not match the background autofluorescence well.  If this is the case, a suitable 'b' or 'c' blank should be chosen instead.

Change 'bl_int' to 1 or 13 according to the blank cycle.  Blank 13 generally has photobleached autofluorescent signal compared with Blank 1, and generally removes more diffuse background.  Sometimes the images from cycles earlier in the run will match closer to blank1 while those later will match closer to blank13.

Sometimes blurring before subtraction can help.  Uncomment to apply to either the blank or signal image.  Adjust sigma to control radius(strength) of blurring.

The first Kview.interact function is for viewing the effects of subtraction on a cropped portion of the image.  Adjust coordinates to determine the size.  The second is for viewing the whole image.  

To assess subtraction, first slowly move the Blank_Percentile slider while noting if autofluorescent structures only are getting dimmer.  Continue until none of these remain while noting if marker signal is being diminished.  Then, slowly move the other slider until signal is brightest without reintroducing autofluorescence/noise.  Once values of Blank_Percentile and Pixel_Cutoff are found, use them for the next Kview.interact to see how they look on the entire image.

Note that these cells are meant to be rerun multiple times if necessary.

### 4.2 First Round

In [12]:
signal_channel = 'CD21'
bl_CH = 'b'
bl_int = 1

params_filename = os.path.join(proc_dir, 'Processing_parameters', f'{signal_channel}_param.txt')
signal_image_tiff = imread(os.path.join(sig_dir, f'{signal_channel}.tif'))
blankID = f'Blank{str(bl_int)}{bl_CH}'
blank_image_tiff = imread(os.path.join(sig_dir, f'{blankID}.tif'))

blank_image_tiff = filters.gaussian(blank_image_tiff, sigma=1, preserve_range=True)
# signal_image_tiff = filters.gaussian(signal_image_tiff, sigma=1, preserve_range=True)

print(signal_channel)

x1 = 3000
x2 = 5000
y1 = 4000
y2 = 7000

# Adjust zoom_factor to fit the image to your screen.  The other values can be adjusted as needed.
Kview.interact(ini_params, signal_image_tiff[y1:y2, x1:x2], blank_image_tiff[y1:y2, x1:x2], zoom_factor=0.2, colormap = 'magma', min_value=0, max_value=50000, step=1000)

# Kview.interact(ini_params, signal_image_tiff, blank_image_tiff, zoom_factor=0.1, colormap = 'viridis', min_value=0, max_value=50000, step=1000)

CD21


In [16]:
Kview.interact(ini_params, signal_image_tiff, blank_image_tiff, zoom_factor=0.1, colormap = 'viridis', min_value=0, max_value=50000, step=1000)

This cell performs the subtraction with values determined above.
If no subtraction was necessary, enter 0 for both and then go to the save cell.

In [13]:
# Enter Pixel_Cutoff from above:
blank_clip_factor = 10000

# Enter Blank_Percentile from above:
background_scale_factor = 1.3

blank_image = np.clip(blank_image_tiff, blank_clip_factor, blank_image_tiff.max())
blank_image[blank_image <= blank_clip_factor] = 0

signal_image = signal_image_tiff - (np.minimum(signal_image_tiff, blank_image * background_scale_factor))

### 4.3 Second Round

If desired the process can be repeated with the same or another blank channel to get satisfactory results. 

In [18]:
print(signal_channel)
bl_CH2 = 'b'
bl_int2 = 1

blankID2 = f'Blank{str(bl_int)}{bl_CH}'
blank_image_tiff_2 = imread(os.path.join(sig_dir, f'{blankID2}.tif'))
blank_image_tiff_2 = filters.gaussian(blank_image_tiff_2, sigma=1, preserve_range=True)

x1 = 3000
x2 = 5000
y1 = 4000
y2 = 7000

Kview.interact(ini_params, signal_image[y1:y2, x1:x2], blank_image_tiff_2[y1:y2, x1:x2], zoom_factor=0.2, colormap = 'viridis', min_value=0, max_value=50000, step=1000)

CD21


In [ ]:
Kview.interact(ini_params, signal_image, blank_image_tiff_2, zoom_factor=0.1, colormap = 'viridis', min_value=0, max_value=50000, step=1000)

In [38]:
blank_clip_factor_2 = 9000
background_scale_factor_2 = 0.4
blank_image_2 = np.clip(blank_image_tiff_2, blank_clip_factor_2, blank_image_tiff_2.max())
blank_image_2[blank_image_2 <= blank_clip_factor_2] = 0
signal_image = signal_image - (np.minimum(signal_image, blank_image_2 * background_scale_factor_2))

## 5. Gaussian Subtraction

### 5.1 Determine Sigma

If no other changes need to be made, go to Section 6 and save.

Sometimes it is helpful to subtract the gaussian of an image to remove patterns of intensity.  This Kview.interact function helps to find the optimal sigma value for blurring.

In [ ]:
Kview.interact(find_gauss, signal_image, zoom_factor=0.1, colormap = 'viridis', min_value=0, max_value=60, step=5)

Enter the sigma value to control the degree of blurring.

In [25]:
sigma = 20
blank = filters.gaussian(signal_image, sigma=sigma, preserve_range=True)

### 5.2 Determine Pixel Range

Apply a range of pixel values to the gaussian subtraction.  Run after performing the gaussian blur in the cell above.

Comment/uncomment the lines to view cropped and full views.

In [ ]:
# Kview.interact(clipped_blank, blank[x1:x2,y1:y2], zoom_factor=0.5, colormap = 'viridis', continuous_update=True, display_min = 0, display_max = None, min_value=0, max_value=np.max(blank), step=100)

Kview.interact(clipped_blank, blank, zoom_factor=0.1, colormap = 'viridis', continuous_update=True, display_min = 0, display_max = None, min_value=0, max_value=np.max(blank), step=100)

### 5.3 Determine Weight

Determine how much subtraction to do, similar to Blank_Percentile in Section 4.  Enter the Gauss_Cutoff values from the cell above.

In [17]:
Kview.interact(final_factor, signal_image, blank,5500,14431, colormap = 'viridis', display_max = None, zoom_factor=.1)

# Kview.interact(final_factor, signal_image[x1:x2,y1:y2], blank[x1:x2,y1:y2], 0,4700, colormap = 'viridis', display_max = None, zoom_factor=.3)

### 5.4 Perform Gaussian Subtraction

Performs the gaussian subtraction.  Enter the values determined above.

In [74]:
g_low = 5300
g_high = 15528
s_image_factor = 0.4

blank[g_low >= blank] = 0  
blank[blank >= g_high] = 0

signal_image_final = signal_image - (np.minimum(signal_image, blank * s_image_factor))

### 5.5 View Results

Check the image before saving.


In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(25, 25))
im = axes.imshow(signal_image_final)
fig.colorbar(im, ax=axes, shrink=.65)
plt.show()

## 6. Save Image and Parameters

Even with this process, it may be that certain markers should be excluded.  Manually create a "failed markers" file to list these and save it in the 'Processing_parameters' folder.

This cell saves the image and the parameters used.  The next steps in FIJI should be saved using Plugins>Macros>Record to record all steps. Copy/paste to append to the params file.


In [39]:
Notes = '''
Here you can add any notes you want to save with the processing parameters.
'''
variables = globals()
variable_names_to_save = ['current_dateTime',
                           'signal_channel', 
                           'blankID', 
                           'blank_clip_factor', 
                           'background_scale_factor', 
                           'blankID2', 
                           'blank_clip_factor_2', 
                           'background_scale_factor_2', 
                           'sigma', 
                           'g_low', 
                           'g_high', 
                           's_image_factor',
                           'blank_factor', 
                           'Notes']

with open(params_filename, 'w') as file:
    for var_name in variable_names_to_save:
        if var_name in variables:
            var_value = variables[var_name]
            var_str = f'{var_name}: {repr(var_value)}\n'
            file.write(var_str)

imsave(os.path.join(proc_dir, f'{signal_channel}.tif'), signal_image)

## 7. Auxilliary Functions

Merge all the images together.

In [7]:
sample_id = '1904_CC2B'
pixelsize = 0.3774
channel_names = []
file_list = glob(os.path.join(proc_dir,'*.tif'))
for file in file_list:

    file_name = os.path.basename(file)
    channel_name = file_name.split('.')[0]
    channel_names.append(channel_name)

image = imread(file_list).astype(np.uint16)

imsave(os.path.join(proc_dir,f'{sample_id}.tif'), image, imagej=True, resolution=(1/pixelsize, 1/pixelsize), resolutionunit='micron', metadata={'axes': 'CYX', 'Labels': channel_names})

This function may be used in place of ini_params above to assess how subtraction affects signal to noise.  Must have a binary segmentation file.

In [ ]:
def ini_params_SNR(images, imageb, seg_mask, blclfa:int=0, bckscfa:float=.0):

    blank_image = np.clip(imageb, blclfa, 65535)
    blank_image[blank_image <= blclfa] = 0

    signal_image = images - (np.minimum(images, blank_image * bckscfa))
    seg_mask = measure.label(seg_mask, return_num=True, background=0)
    properties = measure.regionprops(seg_mask[0], intensity_image=signal_image)
    statistics = {
   
    'area':       [p.area               for p in properties if p.area<800],
    'mean':       [p.mean_intensity     for p in properties if p.area<800]
    }
    df = pd.DataFrame(statistics)
    
    MFI = np.asarray(df['mean'])
    aX = MFI.flatten()
    # compute 20 largest values in aX
    top20 = np.sort(aX)[-20:]
    # compute the mean of bottom 10th percentile of aX
    btm10 = np.sort(aX)[:int(len(aX)*0.1)]
    top20btm10 = np.mean(top20)/np.mean(btm10)
    
    print('SNR is ' + str(top20btm10)) 
    print(df.describe())
    return signal_image

In [ ]:
Kview.interact(ini_params_SNR, signal_image, blank_image_tiff_2, zoom_factor=0.1, colormap = 'viridis', min_value=0, max_value=50000, step=1000)